In [1]:
import numpy as np
import pandas as pd
import re
import sklearn
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
from sklearn.metrics import accuracy_score
import joblib

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [2]:
df=pd.read_csv('train.csv')
df['label']
tensorflow.__version__
print(np.__version__) 

1.19.5


In [3]:
df.head()
df.shape

(20800, 7)

In [4]:
df.dropna()
df['label_numeric'] = pd.to_numeric(df['label'], errors='coerce')
#print(df['label_numeric'])
df_cleaned = df.dropna(subset=['label_numeric'])
independentVar=df_cleaned.drop(['label','label_numeric'],axis=1)
dependentVar=df_cleaned['label_numeric']
independentVar.shape
dependentVar.shape



(20716,)

In [5]:
copy1=independentVar.copy()
copy1.head()
copy1.shape
 

(20716, 6)

In [6]:
from nltk.corpus import stopwords
vocSize=5000

In [7]:
from nltk.stem.porter import PorterStemmer
PS=PorterStemmer()


In [8]:
nltk.download('stopwords')
#type(stopwords.words('english'))
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dietc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dietc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dietc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dietc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
'''
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download required resources
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()
'''

"\nfrom nltk.sentiment.vader import SentimentIntensityAnalyzer\n\n# Download required resources\nnltk.download('vader_lexicon')\nanalyzer = SentimentIntensityAnalyzer()\n"

In [10]:
alpha_re=re.compile('[^a-zA-Z]')
stop_words = set(stopwords.words('english')) - {'is', 'in','not','aint','isnt'}
from tqdm import tqdm


In [11]:
def preprocess(x):
        
    txt= alpha_re.sub(' ',str(x))
    txt = txt.lower() #lowercase the text
    txt = txt.split()
    
    #txt = [analyzer.polarity_scores(j) for j in txt if not j in stopwords.words('english')]  #list comprehension and stemming 
    txt = [lemmatizer.lemmatize(j) for j in txt if not j in stop_words]
    txt= ' '.join(txt)   #string
    return txt
    
def preprocess_zero(row):
    tqdm.pandas()        
    row['clean_text']= row['text'].progress_apply(preprocess)
    return row['clean_text']

    

In [12]:
def preprocess_output(fact):
    return [preprocess(fact)]

In [13]:
corpus=preprocess_zero(copy1)

100%|███████████████████████████████████████████████████████| 20716/20716 [00:34<00:00, 597.67it/s]


In [14]:
#corpus=corpus.tolist()

In [15]:
oneHot=[one_hot(x,vocSize) for x in corpus]
len(corpus)

20716

In [17]:
length=200
embeddedData=tensorflow.keras.preprocessing.sequence.pad_sequences(oneHot,padding='pre',maxlen=length)
print(embeddedData)

[[3716 1532 4393 ... 3953 1244 2567]
 [2433  240 2760 ... 4118  888 2530]
 [1677 3854  809 ... 1804 3268 4055]
 ...
 [3893 1778 3889 ... 3914  666 2841]
 [   0    0    0 ... 4171  885 2426]
 [2867 3321 2619 ... 3983 4472 1439]]


In [18]:
embeddingFeatures=400
model=Sequential()
model.add(Embedding(vocSize,embeddingFeatures,input_length=length))
model.add(Dropout(0.3))
model.add(LSTM(100)) #Adding 100 lstm neurons in the layer
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))


In [19]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 400)          2000000   
_________________________________________________________________
dropout (Dropout)            (None, 200, 400)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               200400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 2,200,501
Trainable params: 2,200,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
dataArray=np.array(embeddedData) 
labelArray=np.asarray(dependentVar).astype(np.float32)


In [21]:
#fit the vectorized data into the lstm model created
dataTrain, dataTest, labelTrain, labelTest = train_test_split(dataArray, labelArray, test_size=0.30,random_state=42)
dataTest.shape

(6215, 200)

In [22]:
# epoch varied
model.fit(dataTrain,labelTrain,validation_data=(dataTest,labelTest),epochs=5,batch_size=64)

Epoch 1/5
227/227 [==============================] - 58s 224ms/step - loss: 0.3403 - accuracy: 0.8500 - val_loss: 0.2067 - val_accuracy: 0.9210
Epoch 2/5
227/227 [==============================] - 50s 222ms/step - loss: 0.1486 - accuracy: 0.9457 - val_loss: 0.1989 - val_accuracy: 0.9228
Epoch 3/5
227/227 [==============================] - 51s 223ms/step - loss: 0.0885 - accuracy: 0.9696 - val_loss: 0.2141 - val_accuracy: 0.9162
Epoch 4/5
227/227 [==============================] - 51s 225ms/step - loss: 0.0619 - accuracy: 0.9797 - val_loss: 0.2831 - val_accuracy: 0.9212
Epoch 5/5
227/227 [==============================] - 51s 225ms/step - loss: 0.0612 - accuracy: 0.9789 - val_loss: 0.3045 - val_accuracy: 0.9199


In [23]:
prediction=(model.predict(dataTest)>0.5).astype("int32")
#LSTM accuracy
accuracy_score(prediction,labelTest)

0.9198712791633146

In [24]:
#Naive Bayes one hot

NB = MultinomialNB()
NB.fit(dataTrain,labelTrain)
predictionNB = NB.predict(dataTest)
accuracy_score(labelTest,predictionNB)

0.6288012872083668

In [25]:
#LR ONEHOT
LR= LogisticRegression(solver='lbfgs',max_iter=700)
LR.fit(dataTrain,labelTrain)
predictionLR = LR.predict(dataTest)
accuracy_score(labelTest,predictionLR)

0.619147224456959

In [26]:
xTrain, xTest, yTrain, yTest = train_test_split(corpus, dependentVar, test_size=0.3,random_state=42)

In [27]:
#compare TDIDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer(max_features=5000) #ngram_range=(1, 3)
tfidTrain = vectorization.fit_transform(xTrain)
tfidTest = vectorization.transform(xTest)

In [28]:
def load_glove(glove_file):
    with open(glove_file, 'r', encoding='utf-8') as f:
        return {line.split()[0]: np.array(line.split()[1:], dtype='float32') for line in f}

def sentence_embedding(sentence, embeddings, dim=100):
    words = sentence.split()
    vectors = [embeddings[word] for word in words if word in embeddings]
    return np.mean(vectors, axis=0) if vectors else np.zeros(dim)

# Load GloVe and prepare embeddings
glove_file = "glove.6B.100d.txt"  
embeddings = load_glove(glove_file)

# Convert train and test sets to embeddings
X_train_embeddings = np.array([sentence_embedding(sent, embeddings) for sent in xTrain])
X_test_embeddings = np.array([sentence_embedding(sent, embeddings) for sent in xTest])


In [29]:
#NB TFIDF
NB2 = MultinomialNB()
NB2.fit(tfidTrain,yTrain)
predictionNB2 = NB2.predict(tfidTest)
accuracy_score(yTest,predictionNB2)


0.8775543041029766

In [30]:
#SVM TFIDF
SVM2 = svm.SVC(C=0.1, kernel='linear', gamma='auto',probability=True,class_weight='balanced')
SVM2.fit(tfidTrain,yTrain)
predictions_SVM2 = SVM2.predict(tfidTest)
accuracy_score(yTest,predictions_SVM2)


0.9316170555108608

In [31]:
#SVM light TFIDF
SVM2_light = svm.LinearSVC(C=0.1,class_weight='balanced')
SVM2_light.fit(tfidTrain,yTrain)
predictions_SVM2_light = SVM2_light.predict(tfidTest)
accuracy_score(yTest,predictions_SVM2_light)
joblib.dump(SVM2_light, 'svm_light.pkl')


['svm_light.pkl']

In [32]:
#SVM light glove TFIDF
SVM2_light_glove = svm.LinearSVC(C=0.1,class_weight='balanced')
SVM2_light_glove.fit(X_train_embeddings,yTrain)
predictions_SVM2_light_glove = SVM2_light_glove.predict(X_test_embeddings)
accuracy_score(yTest,predictions_SVM2_light_glove)
joblib.dump(SVM2_light_glove, 'svm_light_glove.pkl')

['svm_light_glove.pkl']

In [33]:
print(dependentVar.value_counts())  


0.0    10387
1.0    10329
Name: label_numeric, dtype: int64


In [34]:
#LR TFIDF
LR2= LogisticRegression().fit(tfidTrain,yTrain)

predictionLR2 = LR2.predict(tfidTest)
accuracy_score(yTest,predictionLR2)

0.9407884151246984

In [35]:
#Random forest tf idf

from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier(random_state=0)
RFC.fit(tfidTrain,yTrain)
predictionRFC=RFC.predict(tfidTest)

accuracy_score(yTest,predictionRFC)

0.9411102172164119

In [36]:
#Random forest one hot

from sklearn.ensemble import RandomForestClassifier
RFC2=RandomForestClassifier(random_state=0)
RFC2.fit(dataTrain,labelTrain)
predictionRFC2=RFC2.predict(dataTest)

accuracy_score(labelTest,predictionRFC2)

0.7012067578439259

In [37]:
#ada boost tfidf
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

Ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10),n_estimators=5)
Ada.fit(tfidTrain, yTrain)
predictionAda = Ada.predict(tfidTest)
accuracy_score(yTest,predictionAda)

0.8746580852775543

In [38]:
Ada1 = AdaBoostClassifier(RandomForestClassifier(max_depth=10),n_estimators=5)
Ada1.fit(tfidTrain, yTrain)
predictionAda1 = Ada1.predict(tfidTest)
accuracy_score(yTest,predictionAda1)

0.9491552695092518

In [40]:
#loaded_model = joblib.load('svm.pkl')
loaded_model = joblib.load('svm_light.pkl')

statement1="PARIS  —   France chose an idealistic, traditional   candidate in Sunday’s primary to represent the Socialist and   parties in the presidential election this spring. The candidate, Benoît Hamon, 49, who ran on the slogan that he would “make France’s heart beat,” bested Manuel Valls, the former prime minister, whose campaign has promoted more   policies and who has a strong    background. Mr. Hamon appeared to have won by a wide margin, with incomplete returns showing him with an estimated 58 percent of the vote to Mr. Valls’s 41 percent. “Tonight the left holds its head up high again it is looking to the future,” Mr. Hamon said, addressing his supporters. “Our country needs the left, but a modern, innovative left,” he said. Mr. Hamon’s victory was the clearest sign yet that voters on the left want a break with the policies of President François Hollande, who in December announced that he would not seek  . However, Mr. Hamon’s strong showing is unlikely to change widespread assessments that   candidates have little chance of making it into the second round of voting in the general election. The first round of the general election is set for April 23 and the runoff for May 7. The Socialist Party is deeply divided, and one measure of its lack of popular enthusiasm was the relatively low number of people voting. About two million people voted in the second round of the primary on Sunday, in contrast with about 2. 9 million in the second round of the last presidential primary on the left, in 2011. However, much of the conventional wisdom over how the elections will go has been thrown into question over the past week, because the leading candidate, François Fillon, who represents the main   party, the Republicans, was accused of paying his wife large sums of money to work as his parliamentary aide. While nepotism is legal in the French political system, it is not clear that she actually did any work. Prosecutors who specialize in financial malfeasance are reviewing the case. France’s electoral system allows multiple candidates to run for president in the first round of voting, but only the top two   go on to a second round. Mr. Hamon is entering a race that is already crowded on the left, with candidates who include   Mélenchon on the far left, and Emmanuel Macron, an independent who served as economy minister in Mr. Hollande’s government and who embraces more   policies. Unless he decides to withdraw, Mr. Fillon, the mainstream right candidate, will also run, as will the extreme right candidate Marine Le Pen. The two have been expected to go to the runoff. Mr. Hamon’s victory can be attributed at least in part to his image as an idealist and traditional leftist candidate who appeals to union voters as well as more environmentally concerned and socially liberal young people. Unlike Mr. Valls, he also clearly distanced himself from some of Mr. Hollande’s more unpopular policies, especially the economic ones. Thomas Kekenbosch, 22, a student and one of the leaders of the group the Youth With Benoît Hamon, said Mr. Hamon embodied a new hope for those on the left. “We have a perspective we have something to do, to build,” Mr. Kekenbosch said. Mr. Hollande had disappointed many young people because under him the party abandoned ideals, such as support for workers, that many   voters believe in, according to Mr. Kekenbosch. Mr. Hollande’s government, under pressure from the European Union to meet budget restraints, struggled to pass labor code reforms to make the market more attractive to foreign investors and also to encourage French businesses to expand in France. The measures ultimately passed after weeks of strikes, but they were watered down and generated little concrete progress in improving France’s roughly 10 percent unemployment rate and its nearly 25 percent youth joblessness rate. Mr. Hamon strongly endorses a stimulus approach to improving the economy and has promised to phase in a universal income, which would especially help young people looking for work, but would also supplement the livelihood of   French workers. The end goal would be to have everyone receive 750 euros per month (about $840). “We have someone that trusts us,” Mr. Kekenbosch said, “who says: ‘I give you enough to pay for your studies. You can have a scholarship which spares you from working at McDonald’s on provisional contracts for 4 years. ” Mr. Hamon advocates phasing out diesel fuel and encouraging drivers to replace vehicles that use petroleum products with electrical ones. His leftist pedigree began early. His father worked at an arsenal in Brest, a city in the far west of Brittany, and his mother worked off and on as a secretary. He was an early member of the Movement of Young Socialists, and he has continued to work closely with them through his political life. He also worked for Martine Aubry, now the mayor of Lille and a former Socialist Party leader."
statement2="obama was president"
preprocessed_text = preprocess_output(statement1)
print(preprocessed_text)
vector=vectorization.transform(preprocessed_text)

'''
probabilities = loaded_model.predict_proba(vector)  # Use predict_proba to get probabilities
print("Probabilities for False (0) and True (1) classes:", probabilities)
# Get the probability of the 'True' class (class 1)
prob_true_class = probabilities[:, 1]  # Probability for class 1 ('True')
print("Probability of 'True' class:", prob_true_class[0])
'''

output=loaded_model.predict(vector)
if output[0]==1:
    print(f" {output[0]} The text is classified as 'True'.")
else:
    print("The text is classified as 'False'.")


['paris france chose idealistic traditional candidate in sunday primary represent socialist party in presidential election spring candidate beno hamon ran slogan would make france heart beat bested manuel valls former prime minister whose campaign promoted policy strong background mr hamon appeared wide margin incomplete return showing estimated percent vote mr valls percent tonight left hold head high is looking future mr hamon said addressing supporter country need left modern innovative left said mr hamon victory clearest sign yet voter left want break policy president fran ois hollande in december announced would not seek however mr hamon strong showing is unlikely change widespread assessment candidate little chance making second round voting in general election first round general election is set april runoff may socialist party is deeply divided one measure lack popular enthusiasm relatively low number people voting two million people voted in second round primary sunday in cont